In [30]:
import numpy as np
from scipy.optimize import least_squares
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [31]:
# CSVファイルの読み込み
csv_file_path = 'xy.csv'
data = pd.read_csv(csv_file_path)

# xとyのデータを抽出
x_data = data.iloc[:, 0].values  # CSVファイルの1列目をxとして使用
y_data = data.iloc[:, 1].values  # CSVファイルの2列目をyとして使用

In [32]:
# 原子を回転させるための行列
def rotation_matrix():
    identity_matrix = np.identity(3)
    matrices = []
    for i in range(3):
        kx = 0
        ky = 0
        kz = 1
        R = np.array([[0, -kz, ky], [kz, 0, -kx], [-ky, kx, 0]])
        t = 120
        theta = np.radians(t * i)
        rot_mat = identity_matrix + np.sin(theta) * R + (1 - np.cos(theta)) * np.dot(R, R)
        matrices.append(rot_mat)
    return matrices

# Rに書き換えてる
R = rotation_matrix()

# thetaの書き換え
theta = np.radians(x_data)

# ロドリゲスの回転公式×磁場ベクトル
def rodrigues_rotation(n, s, theta):
    h = (s[:, np.newaxis] * np.cos(theta) +
         np.cross(n, s)[:, np.newaxis] * np.sin(theta) +
         n[:, np.newaxis] * np.dot(n, s) * (1 - np.cos(theta)))
    return h

# モデル関数
def model(params, x_data):
    K_rot_values = []
    n = np.array([params[0], params[1], params[2]])
    s = np.array([params[3], params[4], params[5]])
    XX = params[6]
    XY = params[7]
    YY = params[8]
    YZ = params[9]
    XZ = params[10]
    ZZ = params[11]
    K = np.array([[XX, XY, XZ], [XY, YY, YZ], [XZ, YZ, ZZ]])
    h_vec = np.array(h)
    K_rot = np.dot(h_vec.T, np.dot(R[i], np.dot(K, np.dot(np.linalg.inv(R[i]), h_vec))))*10**(4)
    K_rot_values.append(K_rot)
    return K_rot_values

def residual(params, x_data, y_data):
    return model(params, x_data) - y_data